In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import community as community_louvain  
import ast
import random
import tarfile
import json

# Carica dati
edges = pd.read_csv('dataset/spoti/edges.csv')
nodes = pd.read_csv('dataset/spoti/nodes.csv')
nodes_unique = nodes.drop_duplicates(subset=['spotify_id'], keep='first')


In [ ]:
# ============================================================================
# ESTRAZIONE MUSICBRAINZ E MAPPING NAZIONALITÀ
# ============================================================================

def add_nationality_to_nodes(nodes_df):
    """
    Aggiunge la colonna 'nationality' al DataFrame degli artisti usando MusicBrainz.
    """
    
    # DEBUG: Verifica input
    print(f"🔍 DEBUG - Input nodes_df type: {type(nodes_df)}")
    print(f"🔍 DEBUG - Input nodes_df is None: {nodes_df is None}")
    
    if nodes_df is None:
        print("❌ ERRORE: nodes_df è None all'ingresso della funzione!")
        return None
    
    print(f"🔍 DEBUG - nodes_df shape: {nodes_df.shape}")
    print(f"🔍 DEBUG - nodes_df columns: {list(nodes_df.columns)[:5]}...")
    
    print("\nCaricamento artisti da MusicBrainz...")
    artists_dict = {}
    
    # Leggi direttamente dal file artist
    try:
        with open('dataset/spoti/artist/mbdump/artist', 'r', encoding='utf-8') as f:
            count = 0
            matched = 0
            for line in f:
                try:
                    artist = json.loads(line)
                    name = artist.get('name', '').lower().strip()
                    
                    # Prendi il nome del paese dall'oggetto area
                    area = artist.get('area')
                    if area and isinstance(area, dict):
                        # Verifica se è un Country guardando i codici ISO
                        iso_codes = area.get('iso-3166-1-codes', [])
                        if iso_codes:  # Se ha codice ISO paese, è un Country
                            country_name = area.get('name')
                            if name and country_name:
                                if name not in artists_dict:
                                    artists_dict[name] = country_name
                                    matched += 1
                    
                    count += 1
                    if count % 100000 == 0:
                        print(f"  Processati {count} artisti... (matchati: {matched})")
                        
                except Exception as e:
                    continue
        
        print(f"Artisti totali processati: {count}")
        print(f"Artisti con nazionalità estratti: {len(artists_dict)}")
        
    except FileNotFoundError as e:
        print(f"❌ ERRORE: File non trovato - {e}")
        return None
    except Exception as e:
        print(f"❌ ERRORE durante lettura file: {e}")
        return None
    
    # DEBUG: Verifica dizionario
    print(f"\n🔍 DEBUG - artists_dict size: {len(artists_dict)}")
    if len(artists_dict) > 0:
        print(f"🔍 DEBUG - Primi 3 artisti del dizionario:")
        for i, (name, country) in enumerate(list(artists_dict.items())[:3]):
            print(f"    {name} -> {country}")
    
    # Aggiungi nationality al DataFrame
    print(f"\n🔍 DEBUG - Prima di copy(), nodes_df type: {type(nodes_df)}")
    
    try:
        nodes_df = nodes_df.copy()
        print(f"🔍 DEBUG - Dopo copy(), nodes_df shape: {nodes_df.shape}")
        
        # Verifica che esista la colonna 'name'
        if 'name' not in nodes_df.columns:
            print(f"❌ ERRORE: Colonna 'name' non trovata!")
            print(f"   Colonne disponibili: {list(nodes_df.columns)}")
            return None
        
        print(f"🔍 DEBUG - Colonna 'name' trovata, primi 3 valori:")
        print(f"    {nodes_df['name'].head(3).tolist()}")
        
        nodes_df['name_lower'] = nodes_df['name'].str.lower().str.strip()
        print(f"🔍 DEBUG - Creata colonna name_lower, primi 3 valori:")
        print(f"    {nodes_df['name_lower'].head(3).tolist()}")
        
        nodes_df['nationality'] = nodes_df['name_lower'].map(artists_dict)
        print(f"🔍 DEBUG - Creata colonna nationality")
        
        nodes_df.drop(columns=['name_lower'], inplace=True)
        print(f"🔍 DEBUG - Rimossa colonna name_lower")
        
        matched_nodes = nodes_df['nationality'].notna().sum()
        total = len(nodes_df)
        
        print(f"\n{'='*50}")
        print(f"RISULTATI MATCHING")
        print(f"{'='*50}")
        print(f"Artisti matchati: {matched_nodes}/{total} ({100*matched_nodes/total:.1f}%)")
        print(f"Artisti senza nazionalità: {total - matched_nodes}")
        
        print(f"\n🔍 DEBUG - Prima di return, nodes_df type: {type(nodes_df)}")
        print(f"🔍 DEBUG - Prima di return, nodes_df is None: {nodes_df is None}")
        
        return nodes_df
        
    except Exception as e:
        print(f"❌ ERRORE durante processing del DataFrame: {e}")
        import traceback
        traceback.print_exc()
        return None


# DEBUG: Verifica nodes_unique prima della chiamata
print("="*70)
print("VERIFICA PRIMA DELLA CHIAMATA")
print("="*70)
print(f"🔍 nodes_unique type: {type(nodes_unique)}")
print(f"🔍 nodes_unique is None: {nodes_unique is None}")
if nodes_unique is not None:
    print(f"🔍 nodes_unique shape: {nodes_unique.shape}")
    print(f"🔍 nodes_unique columns: {list(nodes_unique.columns)[:5]}...")
print()

# Esegui il mapping
result = add_nationality_to_nodes(nodes_unique)

# DEBUG: Verifica risultato
print("\n" + "="*70)
print("VERIFICA DOPO LA CHIAMATA")
print("="*70)
print(f"🔍 result type: {type(result)}")
print(f"🔍 result is None: {result is None}")
if result is not None:
    print(f"🔍 result shape: {result.shape}")
    print(f"🔍 'nationality' in columns: {'nationality' in result.columns}")
    nodes_unique = result
else:
    print("❌ La funzione ha restituito None!")


In [ ]:
# ============================================================================
# REFINE NATIONALITY USANDO I GENERI
# ============================================================================

def refine_nationality_with_genres(nodes_df):
    """
    Affina la nazionalità degli artisti dando priorità ai generi musicali
    che contengono riferimenti geografici espliciti.
    
    Se un artista ha generi con keyword nazionali, sovrascrive la nationality 
    di MusicBrainz (utile per artisti con nationality mancante o ambigua).
    
    Parametri:
    - nodes_df: DataFrame con colonne 'genres' e 'nationality'
    
    Returns:
    - DataFrame modificato con nationality aggiornata
    """
    
    # Mapping generi -> nazionalità (espandibile)
    GENRE_TO_COUNTRY = {
        # Italia
        "italian": "Italy",
        "ital": "Italy",
        
        # Francia
        "french": "France",
        "fran": "France",
        
        # Germania
        "german": "Germany",
        "deutsch": "Germany",
        
        # Spagna
        "spanish": "Spain",
        
        # UK
        "british": "United Kingdom",
        "uk": "United Kingdom",
        "english": "United Kingdom",
        
        # USA
        "american": "United States",
        
        # Nordici
        "swedish": "Sweden",
        "norwegian": "Norway",
        "danish": "Denmark",
        "finnish": "Finland",
        "icelandic": "Iceland",
        
        # Altri paesi
        "brazilian": "Brazil",
        "portuguese": "Portugal",
        "mexican": "Mexico",
        "argentinian": "Argentina",
        "japanese": "Japan",
        "korean": "South Korea",
        "chinese": "China",
        "indian": "India",
        "turkish": "Turkey",
        "greek": "Greece",
        "polish": "Poland",
        "russian": "Russia",
        "dutch": "Netherlands",
        "belgian": "Belgium",
        "austrian": "Austria",
        "swiss": "Switzerland",
        "canadian": "Canada",
        "australian": "Australia",
        "irish": "Ireland",
        "scottish": "United Kingdom",  # Scozia -> UK
        "welsh": "United Kingdom",      # Galles -> UK
    }
    
    print("Raffinamento nazionalità tramite analisi dei generi...")
    
    nodes_df = nodes_df.copy()
    refined_count = 0
    added_count = 0
    
    for idx, row in nodes_df.iterrows():
        genres = row['genres']
        current_nationality = row['nationality']
        
        # Salta se genres è vuoto o NaN
        if pd.isna(genres) or genres == '[]' or genres == '':
            continue
        
        # Converti stringa di lista in stringa lowercase per il matching
        genres_lower = str(genres).lower()
        
        # Cerca keyword nei generi
        detected_country = None
        for keyword, country in GENRE_TO_COUNTRY.items():
            if keyword in genres_lower:
                detected_country = country
                break  # Prendi il primo match
        
        # Se troviamo una nazionalità nei generi
        if detected_country:
            # Se la nationality è vuota, aggiungiamola
            if pd.isna(current_nationality):
                nodes_df.at[idx, 'nationality'] = detected_country
                added_count += 1
            # Se è diversa da quella di MusicBrainz, diamo priorità ai generi
            elif current_nationality != detected_country:
                nodes_df.at[idx, 'nationality'] = detected_country
                refined_count += 1
    
    print(f"\n{'='*50}")
    print(f"RISULTATI RAFFINAMENTO")
    print(f"{'='*50}")
    print(f"Nazionalità aggiunte (prima assenti): {added_count}")
    print(f"Nazionalità modificate (da MusicBrainz): {refined_count}")
    print(f"Totale artisti con nazionalità: {nodes_df['nationality'].notna().sum()}")
    
    return nodes_df

In [ ]:
# Debug: controlla il formato dei generi
riria = nodes_unique[nodes_unique['name'] == 'Riria.']
print(f"Generi di Riria: {riria['genres'].values}")
print(f"Tipo di dato: {type(riria['genres'].values[0])}")

# Se esiste la riga, stampane anche il contenuto completo
if len(riria) > 0:
    print(f"\nRiga completa:")
    print(riria)
    print(f"\nNazionalità attuale: {riria['nationality'].values[0]}")
else:
    print("❌ Artista 'Riria.' non trovato")


In [ ]:
# Step 2: Raffina usando i generi
print("\n" + "="*70)
print("STEP 2: RAFFINAMENTO CON GENERI MUSICALI")
print("="*70)
nodes_unique = refine_nationality_with_genres(nodes_unique)

In [ ]:
# Correzione: devi iterare sulle righe del DataFrame
for idx, row in nodes_unique.iterrows():
    if row['name'] == "MACE":
        print(row)


In [ ]:
# Salva nodes_unique come nuovo nodes.csv
nodes_unique.to_csv('dataset/spoti/nodes.csv', index=False)
print("✓ File salvato: dataset/spoti/nodes.csv")
print(f"  Shape: {nodes_unique.shape}")
print(f"  Colonne: {list(nodes_unique.columns)}")
print(f"  Artisti con nazionalità: {nodes_unique['nationality'].notna().sum()}")
